# viz

> Visulaization modules

In [ ]:
#| default_exp viz

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import warnings
warnings.simplefilter("ignore", UserWarning)
import seaborn

import numpy as np
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import pandas as pd
import sepia.SepiaPlot as SepiaPlot
from sepia.SepiaModel import SepiaModel
from sepia.SepiaData import SepiaData
from matplotlib import ticker
from itertools import cycle
import matplotlib.colors as mcolors
import matplotlib.cm as cm

In [ ]:
#| export

def plot_lines_with_param_color(param_array:np.array=None, # parameter array
                                x_array:np.array=None, # x-axis array
                                y_array_all:np.array=None, # y-axis array
                                title_str:str=None, # Title string
                                xlabel_str:str=None, # x-label string
                                ylabel_str:str=None, # y-label string
                                param_name_str:str=None, # Parameter string
                               ):
    
    
    f, a = plt.subplots(1,1, figsize = (8, 5))  

    norm = mpl.colors.Normalize(vmin=param_array.min(), vmax=param_array.max())
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
    cmap.set_array([])

    for sim_index in range(param_array.shape[0]):
        a.plot(x_array, y_array_all[sim_index], 
                 '-', alpha= 0.5, c=cmap.to_rgba(param_array[sim_index]), 
                 label='Sim: '+str(sim_index) )

    # plt.plot(stellar_mass_test, gsmf_um_test, 'k.', label='UM z=0.00')
    a.set_xscale('log')
    # plt.axhline(y=0, linestyle='dashed', color='black')
    # plt.yscale('log')
    # plt.xlim(4e9, )
    # plt.ylim(-0.02, 0.02)

    clb = f.colorbar(cmap , ax=a)

    clb.ax.tick_params(labelsize=15) 
    clb.ax.set_title(param_name_str, fontsize=15)

    plt.title(title_str, fontsize=15)
    a.set_xlabel(xlabel_str, fontsize=15)
    a.set_ylabel(ylabel_str, fontsize=15)
    
    return f

In [ ]:
#| export

def plot_scatter_matrix(df:pd.DataFrame=None, 
                        colors:str=None,
                       ): 
    
    f, a = plt.subplots(1,1, figsize = (10, 10))
    scatter_matrix = pd.plotting.scatter_matrix(df, 
                                                color=colors,  
                                                figsize=(10,10), 
                                                alpha=1.0, 
                                                ax=a, 
                                                grid=False, 
                                                diagonal='hist',
                                                range_padding=0.1,
                                                s=80);

    for ax in scatter_matrix.ravel():
        ax.set_xlabel(ax.get_xlabel(), fontsize = 14, rotation = 0)
        ax.set_ylabel(ax.get_ylabel(), fontsize = 14, rotation = 90)
        
    return f

In [ ]:
#| export

def plot_train_diagnostics(sepia_model:SepiaModel=None, # Input data in SEPIA format, after PCA
                          ) -> tuple: #Pair-plot and Trace-plot
    samples_dict = sepia_model.get_samples()
    theta_pairs = SepiaPlot.theta_pairs(samples_dict)
    mcmc_trace = SepiaPlot.mcmc_trace(samples_dict)
    
    return theta_pairs, mcmc_trace

In [ ]:
#| export

def sensitivity_plot(k_all, # all wavenumbers
                     params_all, # all parameters
                     sepia_model, # SEPIA emulator model
                     emulator_function, # function which takes in sepia model and parameters
                     param_name, # Parameter name
                    ):

    color_by_index = 0
    colorparams = params_all[:, color_by_index]
    # colorparams = X_test_transformed1[:, color_by_index]
    colormap = cm.Dark2
    normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))

    allMax = np.max(params_all, axis = 0)
    allMin = np.min(params_all, axis = 0)
    allMean = np.mean(params_all, axis = 0)

    numPlots = 100

    fig, ax = plt.subplots(5,1, figsize = (7, 15), sharex='col')
    plt.subplots_adjust(wspace=0.25)
    plt.subplots_adjust(hspace=0.05)
    plt.suptitle('Sensitivity analysis using emulator', fontsize=18, y=0.95)


    for paramNo in range(5):
            para_range = np.linspace(allMin[paramNo], allMax[paramNo], numPlots)        
            # colorList = plt.cm.coolwarm(np.linspace(0,1,numPlots))

            colormap = cm.coolwarm
            normalize = mcolors.Normalize(vmin=np.min(allMin[paramNo]), vmax=allMax[paramNo])

            for plotID in range(numPlots):
                    para_plot = np.copy(allMean)
                    para_plot[paramNo] = para_range[plotID]  #### allMean gets changed everytime!!

                    color = colormap(normalize(para_plot[paramNo]))

                    gsmf_decoded, _ = emulator_function(sepia_model, para_plot)

                    lineObj = ax[paramNo].plot(k_all, gsmf_decoded, lw= 1.5, color = color) 

                    # ax[paramNo].set_yscale('log')
                    ax[paramNo].set_xscale('log')
                    ax[paramNo].set_ylabel('B(k)', fontsize=18)
                    ax[paramNo].set_yticks([], minor = True)
                            
            
            # Colorbar setup
            s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
            s_map.set_array(colorparams)

            # If color parameters is a linspace, we can set boundaries in this way
            halfdist = (colorparams[1] - colorparams[0])/2.0
            boundaries = np.linspace(colorparams[0] - halfdist, colorparams[-1] + halfdist, len(colorparams) + 1)

            cbar = fig.colorbar(s_map, spacing='proportional', ax=ax[paramNo])

            cbarlabel = param_name[paramNo]
            cbar.set_label(cbarlabel, fontsize=20)

    ax[paramNo].set_xlabel('k[h/Mpc]', fontsize=18)
    # plt.show()
    
    return fig

In [ ]:
#| export 

def validation_plot(k_all, target_vals, pred_mean, pred_quant):

    f, a = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios': [2, 1]}, sharex=True)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.05)

    colors = ['b', 'r', 'g', ]
    styles = ['-', '--', '-.']
    styles_label = ['Hi-COLA', 'Emulated mean', 'Emulated (0.05, 0.95) quantile']


    for one_index in range(pred_mean.shape[1]):

        a[0].plot(k_all, target_vals[one_index], c=colors[one_index], ls=styles[0])
        a[0].plot(k_all, pred_mean[:, one_index], c=colors[one_index], ls=styles[1])
        a[0].plot(k_all, pred_quant[:, one_index, 0], c=colors[one_index], ls=styles[2])

        a[1].plot(k_all, (pred_mean[:, one_index]/target_vals[one_index]) - 1, c=colors[one_index])

    for cc, col in enumerate(colors):
        a[0].plot(np.NaN, np.NaN, c=colors[cc], label='Cosmology: ' + str(cc))

    ax2 = a[0].twinx()
    for ss, sty in enumerate(styles):
        ax2.plot(np.NaN, np.NaN, ls=styles[ss],
                 label=styles_label[ss], c='black')

    ax2.get_yaxis().set_visible(False)
    
    a[0].fill_between(k_all, 0.83, 1.1, where=(k_all > 1.2), color='k', alpha=0.15)
    a[1].fill_between(k_all, -0.012, 0.012, where=(k_all > 1.2), color='k', alpha=0.15)


    a[0].legend(loc=1, title='Test configuration')
    ax2.legend(loc=3)
    a[1].set_xlabel('k[h/Mpc]')
    a[1].set_ylabel(r'$\delta B(k)/B(k)$')
    a[0].set_ylabel('B(k)')
    a[0].set_xscale('log')
    # plt.show()
    a[0].set_xlim(2e-2, 1e1)
    a[0].set_ylim(0.83, 1.1)
    a[1].set_ylim(-0.012, 0.012)
    
    return f

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()